# Evaluating and Improving Models

Training a deep learning model is an art, and to make sure our model is trained correctly, we need to keep track of certain metrics during training, such as the loss or the accuracy. We will learn how to calculate such metrics and how to reduce overfitting using an image dataset as an example.

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

C:\Users\jcamp\AppData\Local\Temp\ipykernel_15428\2178698562.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## A deeper dive into loading data

### Using the TensorDataset class
In practice, loading your data into a PyTorch dataset will be one of the first steps you take in order to create and train a neural network with PyTorch.

The TensorDataset class is very helpful when your dataset can be loaded directly as a NumPy array. Recall that `TensorDataset()` can take one or more NumPy arrays as input.

In this exercise, you'll practice creating a PyTorch dataset using the TensorDataset class.

Instructions:

- Convert the NumPy arrays provided to PyTorch tensors.
- Create a TensorDataset using the `torch_features` and the `torch_target` tensors provided (in this order).
- Return the last element of the dataset.

In [5]:
np_features = np.array(np.random.rand(12, 8))
np_target = np.array(np.random.rand(12, 1))

# Convert arrays to PyTorch tensors
torch_features = torch.tensor(np_features)
torch_target = torch.tensor(np_target)

# Create a TensorDataset from two tensors
dataset = TensorDataset(torch_features, torch_target)

# Return the last element of this dataset
print(dataset[-1])

(tensor([0.3122, 0.0692, 0.8436, 0.7042, 0.3693, 0.0211, 0.2532, 0.6322],
       dtype=torch.float64), tensor([0.6663], dtype=torch.float64))


### From data loading to running a forward pass

In this exercise, you'll create a PyTorch `DataLoader` from a pandas DataFrame and call a model on this dataset. Specifically, you'll run a forward pass on a neural network. You'll continue working with fully connected neural networks, as you have done so far.

You'll begin by subsetting a loaded DataFrame called `dataframe`, converting features and targets NumPy arrays, and converting to PyTorch tensors in order to create a PyTorch dataset.

This dataset can be loaded into a PyTorch DataLoader, batched, shuffled, and used to run a forward pass on a custom fully connected neural network.

Instructions:

- Extract the features (`ph`, `Sulfate`, `Conductivity`, `Organic_carbon`) and target (`Potability`) values and load them into the appropriate tensors to represent features and targets.
- Use both tensors to create a PyTorch dataset using the dataset class that's quickest to use when tensors don't require any additional preprocessing.
- Create a PyTorch DataLoader from the created `TensorDataset`; this DataLoader should use a `batch_size` of two and `shuffle` the dataset.
- Implement a small, fully connected neural network using exactly two linear layers and the `nn.Sequential()` API, where the final output size is 1.

In [10]:
dataframe = pd.read_csv("datasets/water_potability.csv")
dataframe.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,0.587349,0.577747,0.386298,0.568199,0.647347,0.292985,0.654522,0.795029,0.630115,0
1,0.643654,0.441300,0.314381,0.439304,0.514545,0.356685,0.377248,0.202914,0.520358,0
2,0.388934,0.470876,0.506122,0.524364,0.561537,0.142913,0.249922,0.401487,0.219973,0
3,0.725820,0.715942,0.506141,0.521683,0.751819,0.148683,0.467200,0.658678,0.242428,0
4,0.610517,0.532588,0.237701,0.270288,0.495155,0.494792,0.409721,0.469762,0.585049,0


In [13]:
dataframe[["ph", "Sulfate", "Conductivity", "Organic_carbon"]].to_numpy()

array([[0.58734916, 0.64734744, 0.29298545, 0.65452157],
       [0.64365393, 0.51454537, 0.35668464, 0.37724796],
       [0.38893354, 0.56153738, 0.14291265, 0.24992171],
       ...,
       [0.81782618, 0.36908889, 0.43187239, 0.56326524],
       [0.42418706, 0.61557214, 0.38836022, 0.39778031],
       [0.32242529, 0.65604679, 0.58870938, 0.47142165]])

In [18]:
# Load the different columns into two PyTorch tensors
features = torch.tensor(
    dataframe[["ph", "Sulfate", "Conductivity", "Organic_carbon"]].to_numpy()
).float()
target = torch.tensor(dataframe["Potability"].to_numpy()).float()

# Create a dataset from the two generated tensors
dataset = TensorDataset(features, target)

# Create a dataloader using the above dataset
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
x, y = next(iter(dataloader))

# Create a model using the nn.Sequential API
model = nn.Sequential(nn.Linear(4, 2), nn.Linear(2, 1))
output = model(features)
print(output)

tensor([[0.6303],
        [0.5075],
        [0.4814],
        ...,
        [0.5174],
        [0.5669],
        [0.6336]], grad_fn=<AddmmBackward0>)


## Evaluating model performance

### Writing the evaluation loop

In this exercise, you will practice writing the evaluation loop. Recall that the evaluation loop is similar to the training loop, except that you will not perform the gradient calculation and the optimizer step.


Instructions:
- Set the model to evaluation mode.
- Sum the current batch loss to the validation_loss variable.
- Calculate the mean loss value for the epoch.
- Set the model back to training mode.

In [ ]:
# Set the model to evaluation mode
model.eval()
validation_loss = 0.0

with torch.no_grad():

    for data in validationloader:

        outputs = model(data[0])
        loss = criterion(outputs, data[1])

        # Sum the current loss to the validation_loss variable
        validation_loss += loss.item()

# Calculate the mean loss value
validation_loss_epoch = validation_loss / len(validationloader)
print(validation_loss_epoch)

# Set the model back to training mode
model.train()

### Calculating accuracy using torchmetrics

In addition to the losses, you should also be keeping track of the accuracy during training. By doing so, you will be able to select the epoch when the model performed the best.

In this exercise, you will practice using the `torchmetrics` package to calculate the accuracy. You will be using a sample of the facemask dataset. This dataset contains three different classes. The `plot_errors` function will display samples where the model predictions do not match the ground truth. Performing such error analysis will help you understand your model failure modes.

The `torchmetrics` package is already imported. The model outputs are the probabilities returned by a softmax as the last step of the model. The labels tensor contains the labels as one-hot encoded vectors.

Instructions:
- Create an accuracy metric for a "multiclass" problem with three classes.
- Calculate the accuracy for each batch of the dataloader.
- Calculate accuracy for the epoch.
- Reset the metric for the next epoch.

In [ ]:
# Create accuracy metric using torch metrics
metric = torchmetrics.Accuracy(task="multiclass", num_classes=3)
for data in dataloader:
    features, labels = data
    outputs = model(features)

    # Calculate accuracy over the batch
    acc = metric(outputs.softmax(dim=-1), labels.argmax(dim=-1))

# Calculate accuracy over the whole epoch
acc = metric.compute()

# Reset the metric for the next epoch
metric.reset()
plot_errors(model, dataloader)

## Fighting overfitting


### Experimenting with dropout

The dropout layer randomly zeroes out elements of the input tensor. Doing so helps fight overfitting. In this exercise, you'll create a small neural network with at least two linear layers, two dropout layers, and two activation functions.

Instructions:

- Create a small neural network with one linear layer, one ReLU function, and one dropout layer, in that order. The model should take input_tensor as input and return an output of size 16.
- Using the same neural network, set the probability of zeroing out elements in the dropout layer to 0.8.

In [ ]:
# Create a small neural network
model = nn.Sequential(nn.Linear(3072, 16),
                      nn.ReLU(),
                      nn.Dropout())
model(input_tensor)

In [ ]:
# Create a small neural network
model = nn.Sequential(nn.Linear(3072, 16), nn.ReLU(), nn.Dropout(p=0.8))
model(input_tensor)

## Improving model performance

### Implementing random search

Hyperparameter search is a computationally costly approach to experiment with different hyperparameter values. However, it can lead to performance improvements. In this exercise, you will implement a random search algorithm.

You will randomly sample 10 values of the learning rate and momentum from the uniform distribution. To do so, you will use the `np.random.uniform()` function.

Instructions:

- Randomly sample a learning rate factor between `2` and `4` so that the learning rate (`lr`) is bounded between `10^-2` and `10^-4`
- Randomly sample a momentum between 0.85 and 0.99.

In [ ]:
values = []
for idx in range(10):
    # Randomly sample a learning rate factor between 2 and 4
    factor = np.random.uniform(2, 4)
    lr = 10**-factor

    # Randomly select a momentum between 0.85 and 0.99
    momentum = np.random.uniform(0.85, 0.99)

    values.append((lr, momentum))